In [1]:
%pip install kafka-python avro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 KB 1.3 MB/s eta 0:00:001.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB 1.4 MB/s eta 0:00:002.1 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for avro: filename=avro-1.11.3-py2.py3-none-any.whl size=123913 sha256=283a925da0ea56db431c434f2a1c7051d0f7be77cb6d097a96f6c88d1db91410
  Stored in directory: /Users/supawit/Library/Caches/pip/wheels/1d/f6/41/0e0399396af07060e64d4e32c8bd259b48b98a4a114df31294
Successfully built avro
You should consider upgrading via the '/Users/supawit/.pyenv/versions/3.10.4/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [4]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [5]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [6]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [7]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [8]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [9]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [10]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [11]:
vid = "V489467"
token = "5e7a2120980f0e4737cf8906867c5e9f"

In [18]:
result = None
for message in txconsumer:
    tx = message.value
    print(tx)
    submit = {
        'vid': vid,
        'txid': tx['txid'],
        'signature': gen_signature(tx['txid'], tx['payer'], tx['payee'], tx['amount'], token)
    }
    print(submit)
    producer.send('submit', serialize(submitschema, submit))
    for message in resultconsumer:
        result = message.value
        if result['txid'] == tx['txid']:
            break
    if result:
        print(result)
        break
    else:
        print("No result for txid", tx['txid'])

{'txid': 'TX08426', 'payer': 'A01940', 'payee': 'A16263', 'amount': 1045}
{'vid': 'V489467', 'txid': 'TX08426', 'signature': '227f2beb34c4aa05de58d60ae1f9bcd8'}
{'timestamp': 1711292352, 'vid': 'V489467', 'txid': 'TX08426', 'checksum': '7186bfb6ea58c1e4d9a6e58b7dcdb072', 'code': 200, 'message': 'Confirm'}
